<a href="https://colab.research.google.com/github/rizkiar00/Bagging-Berbasis-Naive-Bayes/blob/master/Bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [247]:
!git clone https://github.com/rizkiar00/Bagging-Berbasis-Naive-Bayes

fatal: destination path 'Bagging-Berbasis-Naive-Bayes' already exists and is not an empty directory.


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt; 
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [442]:
train = np.genfromtxt('Bagging-Berbasis-Naive-Bayes/TrainsetTugas4ML.csv', delimiter=',', skip_header=1)
dftrain = pd.DataFrame(data=train)
print("Kelas:", pd.Series(dftrain.loc[:,2], name='A').unique())
dftrain[2].value_counts()



Kelas: [2. 1.]


1.0    220
2.0     78
Name: 2, dtype: int64

In [528]:
jumlahmodel = 10
array = np.arange(0,dftrain.shape[0])
random = np.random.randint(dftrain.shape[0], size=(jumlahmodel,75))
test = [i for i in array if i not in random]
print("Jumlah Sisa: ",len(test))
Xtrainrandom = []
Ytrainrandom = []
for i in range(jumlahmodel):
  Ytrainrandom.append(np.delete(train[random[i]], (0,1), axis=1)) 
  Xtrainrandom.append(np.delete(train[random[i]], 2, axis=1)) 

objmodel = list()
for i in range(jumlahmodel):
    objmodel.append(GaussianNB().fit(Xtrainrandom[i],Ytrainrandom[i].ravel()))

Ytestsisa = np.delete(train[test], (0,1), axis=1)
Xtestsisa = np.delete(train[test], 2, axis=1)
Ytestpredict = []
for i in range(jumlahmodel):
  Ytestpredict.append(objmodel[i].predict(Xtestsisa))
  print("Akurasi Model {}: {}".format(i,np.mean(Ytestpredict[i] == Ytestsisa)))
  
BaggingYtestpredict = []
for i in range(len(Ytestsisa)):
  votebagging = np.array(Ytestpredict)[:,i]
  votebagging = votebagging.astype(int)
  counts = np.bincount(votebagging)
  BaggingYtestpredict.append(np.argmax(counts))
print("Akurasi Model Bagging: {}".format(np.mean(BaggingYtestpredict == np.transpose(Ytestsisa))))

Jumlah Sisa:  20
Akurasi Model 0: 0.54
Akurasi Model 1: 0.56
Akurasi Model 2: 0.58
Akurasi Model 3: 0.58
Akurasi Model 4: 0.56
Akurasi Model 5: 0.54
Akurasi Model 6: 0.58
Akurasi Model 7: 0.56
Akurasi Model 8: 0.56
Akurasi Model 9: 0.54
Akurasi Model Bagging: 0.95


In [536]:
confusion_df = pd.DataFrame(confusion_matrix(Ytestsisa,BaggingYtestpredict),
             columns=["Predicted Class " + str(class_name) for class_name in [1.0,2.0]],
             index = ["Class " + str(class_name) for class_name in [1.0,2.0]])
print(confusion_df)

           Predicted Class 1.0  Predicted Class 2.0
Class 1.0                   13                    1
Class 2.0                    0                    6


In [537]:
BaggingYtestpredict

[2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1]

In [539]:
 np.transpose(Ytestsisa)

array([[2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
        1., 1., 1., 1.]])